In [1]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

Loading required package: Rsamtools

Loading required package: GenomeInfoDb

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: Biostrings



Warning message:
“no DISPLAY variable so Tk is not available”
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Loaded TSdist v3.7.1. See ?TSdist for help, citation("TSdist") for use in publication.


Loading required package: dtplyr

Loading required package: profvis

Loading required package: ggforce

Loading required package: pracma


Attaching package: ‘pracma’


The following object is masked from ‘package:purrr’:

    cross


The following object is masked from ‘package:Hmisc’:

    ceil


Loading required package: rstatix


Attaching package: ‘rstatix’


The following object is masked from ‘package:IRanges’:

    desc


The following object is masked from ‘package:stats’:

    filter


Loading required package: ggpubr

Loading required package: jsonlite


Attaching package: ‘jsonlite’


The following object is masked from ‘package:R.utils’:

    validate


The following object is masked from ‘package:purrr’:

    flatten


Loading

In [2]:
DIR_IN="../../output/mapAspergillus/"
samples_metadata <- read.csv(file = "../../output/samples_metadata.csv") %>% as.data.table()
list.files(DIR_IN, pattern = "[0-9]_.*_R(1)(pos|neg)_TLEN_EndMotif.txt") %>% tail()
list.files(DIR_IN, pattern = "___.*_R(1)(pos|neg)_TLEN_EndMotif.txt") %>% tail()

[1] "NTC2Run4_EPRSc2_conf0.4_746128_AtubingensisCBS134.48_R1neg_TLEN_EndMotif.txt"
[2] "NTC2Run4_EPRSc2_conf0.4_746128_AtubingensisCBS134.48_R1pos_TLEN_EndMotif.txt"
[3] "NTC2Run4_EPRSc2_conf0.4_746128_AversicolorCBS583.65_R1neg_TLEN_EndMotif.txt" 
[4] "NTC2Run4_EPRSc2_conf0.4_746128_AversicolorCBS583.65_R1pos_TLEN_EndMotif.txt" 
[5] "NTC2Run4_EPRSc2_conf0.4_746128_AwentiiDTO134E9_R1neg_TLEN_EndMotif.txt"      
[6] "NTC2Run4_EPRSc2_conf0.4_746128_AwentiiDTO134E9_R1pos_TLEN_EndMotif.txt"

[1] "H35Pctrl_unmappedHost___AtubingensisCBS134.48_R1neg_TLEN_EndMotif.txt"
[2] "H35Pctrl_unmappedHost___AtubingensisCBS134.48_R1pos_TLEN_EndMotif.txt"
[3] "H35Pctrl_unmappedHost___AversicolorCBS583.65_R1neg_TLEN_EndMotif.txt" 
[4] "H35Pctrl_unmappedHost___AversicolorCBS583.65_R1pos_TLEN_EndMotif.txt" 
[5] "H35Pctrl_unmappedHost___AwentiiDTO134E9_R1neg_TLEN_EndMotif.txt"      
[6] "H35Pctrl_unmappedHost___AwentiiDTO134E9_R1pos_TLEN_EndMotif.txt"

In [3]:
if (exists("Asp_IS_META")){rm("Asp_IS_META")}
#profvis({
for (file in list.files(DIR_IN, pattern = "1131492_.*_R(1|2)(pos|neg)_TLEN_EndMotif.txt")){
    print(file)
    sample_tmp <- unlist(strsplit(file, "_"))[1] 
    taxID_tmp <- unlist(strsplit(file, "_"))[4]
    mapGenome_tmp <- unlist(strsplit(file, "_"))[5]
    side_tmp <- unlist(strsplit(file, "_"))[6]
    
    if(countLines(paste0(DIR_IN, file)) > 1){
        pre_tmp_IS <- fread(paste0(DIR_IN, file), sep = ",") %>% 
                    dplyr::rename(EndMotif_tmp = EndMotif) %>% 
                    mutate(sample_id = sample_tmp) %>% 
                    mutate(side = side_tmp) %>% 
                    mutate(taxID = taxID_tmp) %>% 
                    mutate(mapGenome = mapGenome_tmp) %>% 
                    mutate(read = substr(side_tmp, 0, 2)) %>% 
                    filter(TLEN != 0) %>% #remove TLEN not equal to 0
                    filter(abs(TLEN) < 1000) %>% #filter max TLEN of 1000
                    filter(!grepl(EndMotif_tmp, pattern = "N")) %>% #remove reads with end-motif including "N"
                    group_by(TLEN, EndMotif_tmp, sample_id, side, taxID, mapGenome) %>% 
                    summarise(Count = sum(Count), .groups = "keep")
        
        if (grepl(side_tmp, pattern = "neg")){
            tmp_IS <- pre_tmp_IS 
            tmp_IS$EndMotif <- as.character(reverseComplement(DNAStringSet(pre_tmp_IS$EndMotif_tmp))) 
        } else {
            tmp_IS <- pre_tmp_IS 
            tmp_IS$EndMotif <- pre_tmp_IS$EndMotif_tmp
        }
        
        tmp_IS_meta <- merge(x = samples_metadata, y = tmp_IS, by.x = "sample_id", by.y = "sample_id") 
        if (!exists("Asp_IS_META")){Asp_IS_META <- tmp_IS_meta} else {
            Asp_IS_META <- rbind(Asp_IS_META, tmp_IS_meta)}
    }
    if (exists("Asp_IS_META")){
        fwrite(Asp_IS_META, "../../output/PP_figures/1131492_IS_meta.csv", row.names = FALSE)
    }
}
#})

[1] "A01Basp_EPRSc2_conf0.4_1131492_AaculeatusATCC16872_R1neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_1131492_AaculeatusATCC16872_R1pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_1131492_AaculeatusATCC16872_R2neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_1131492_AaculeatusATCC16872_R2pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_1131492_AbrasiliensisCBS101740_R1neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_1131492_AbrasiliensisCBS101740_R1pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_1131492_AbrasiliensisCBS101740_R2neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_1131492_AbrasiliensisCBS101740_R2pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_1131492_AcampestrisIBT28561_R1neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_1131492_AcampestrisIBT28561_R1pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_1131492_AcampestrisIBT28561_R2neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_1131492_AcampestrisIBT28561_R2pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPR

In [5]:
if (exists("Asp_IS_META")){rm("Asp_IS_META")}
#profvis({
for (file in list.files(DIR_IN, pattern = "746128_.*_R(1|2)(pos|neg)_TLEN_EndMotif.txt")){
    print(file)
    sample_tmp <- unlist(strsplit(file, "_"))[1] 
    taxID_tmp <- unlist(strsplit(file, "_"))[4]
    mapGenome_tmp <- unlist(strsplit(file, "_"))[5]
    side_tmp <- unlist(strsplit(file, "_"))[6]
    
    if(countLines(paste0(DIR_IN, file)) > 1){
        pre_tmp_IS <- fread(paste0(DIR_IN, file), sep = ",") %>% 
                    dplyr::rename(EndMotif_tmp = EndMotif) %>% 
                    mutate(sample_id = sample_tmp) %>% 
                    mutate(side = side_tmp) %>% 
                    mutate(taxID = taxID_tmp) %>% 
                    mutate(mapGenome = mapGenome_tmp) %>% 
                    mutate(read = substr(side_tmp, 0, 2)) %>% 
                    filter(TLEN != 0) %>% #remove TLEN not equal to 0
                    filter(abs(TLEN) < 1000) %>% #filter max TLEN of 1000
                    filter(!grepl(EndMotif_tmp, pattern = "N")) %>% #remove reads with end-motif including "N"
                    group_by(TLEN, EndMotif_tmp, sample_id, side, taxID, mapGenome) %>% 
                    summarise(Count = sum(Count), .groups = "keep")
        
        if (grepl(side_tmp, pattern = "neg")){
            tmp_IS <- pre_tmp_IS 
            tmp_IS$EndMotif <- as.character(reverseComplement(DNAStringSet(pre_tmp_IS$EndMotif_tmp))) 
        } else {
            tmp_IS <- pre_tmp_IS 
            tmp_IS$EndMotif <- pre_tmp_IS$EndMotif_tmp
        }
        
        tmp_IS_meta <- merge(x = samples_metadata, y = tmp_IS, by.x = "sample_id", by.y = "sample_id") 
        if (!exists("Asp_IS_META")){Asp_IS_META <- tmp_IS_meta} else {
            Asp_IS_META <- rbind(Asp_IS_META, tmp_IS_meta)}
    }
    if (exists("Asp_IS_META")){
        fwrite(Asp_IS_META, "../../output/PP_figures/746128_IS_meta.csv", row.names = FALSE)
    }
}
#})

[1] "A01Basp_EPRSc2_conf0.4_746128_AaculeatusATCC16872_R1neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_746128_AaculeatusATCC16872_R1pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_746128_AaculeatusATCC16872_R2neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_746128_AaculeatusATCC16872_R2pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_746128_AbrasiliensisCBS101740_R1neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_746128_AbrasiliensisCBS101740_R1pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_746128_AbrasiliensisCBS101740_R2neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_746128_AbrasiliensisCBS101740_R2pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_746128_AcampestrisIBT28561_R1neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_746128_AcampestrisIBT28561_R1pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_746128_AcampestrisIBT28561_R2neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_746128_AcampestrisIBT28561_R2pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_

In [6]:
if (exists("Asp_IS_META")){rm("Asp_IS_META")}
#profvis({
for (file in list.files(DIR_IN, pattern = "5052_.*_R(1|2)(pos|neg)_TLEN_EndMotif.txt")){
    print(file)
    sample_tmp <- unlist(strsplit(file, "_"))[1] 
    taxID_tmp <- unlist(strsplit(file, "_"))[4]
    mapGenome_tmp <- unlist(strsplit(file, "_"))[5]
    side_tmp <- unlist(strsplit(file, "_"))[6]
    
    if(countLines(paste0(DIR_IN, file)) > 1){
        pre_tmp_IS <- fread(paste0(DIR_IN, file), sep = ",") %>% 
                    dplyr::rename(EndMotif_tmp = EndMotif) %>% 
                    mutate(sample_id = sample_tmp) %>% 
                    mutate(side = side_tmp) %>% 
                    mutate(taxID = taxID_tmp) %>% 
                    mutate(mapGenome = mapGenome_tmp) %>% 
                    mutate(read = substr(side_tmp, 0, 2)) %>% 
                    filter(TLEN != 0) %>% #remove TLEN not equal to 0
                    filter(abs(TLEN) < 1000) %>% #filter max TLEN of 1000
                    filter(!grepl(EndMotif_tmp, pattern = "N")) %>% #remove reads with end-motif including "N"
                    group_by(TLEN, EndMotif_tmp, sample_id, side, taxID, mapGenome) %>% 
                    summarise(Count = sum(Count), .groups = "keep")
        
        if (grepl(side_tmp, pattern = "neg")){
            tmp_IS <- pre_tmp_IS 
            tmp_IS$EndMotif <- as.character(reverseComplement(DNAStringSet(pre_tmp_IS$EndMotif_tmp))) 
        } else {
            tmp_IS <- pre_tmp_IS 
            tmp_IS$EndMotif <- pre_tmp_IS$EndMotif_tmp
        }
        
        tmp_IS_meta <- merge(x = samples_metadata, y = tmp_IS, by.x = "sample_id", by.y = "sample_id") 
        if (!exists("Asp_IS_META")){Asp_IS_META <- tmp_IS_meta} else {
            Asp_IS_META <- rbind(Asp_IS_META, tmp_IS_meta)}
    }
    if (exists("Asp_IS_META")){
        fwrite(Asp_IS_META, "../../output/PP_figures/5052_IS_meta.csv", row.names = FALSE)
    }
}
#})

[1] "A01Basp_EPRSc2_conf0.4_5052_AaculeatusATCC16872_R1neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AaculeatusATCC16872_R1pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AaculeatusATCC16872_R2neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AaculeatusATCC16872_R2pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AbrasiliensisCBS101740_R1neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AbrasiliensisCBS101740_R1pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AbrasiliensisCBS101740_R2neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AbrasiliensisCBS101740_R2pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AcampestrisIBT28561_R1neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AcampestrisIBT28561_R1pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AcampestrisIBT28561_R2neg_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AcampestrisIBT28561_R2pos_TLEN_EndMotif.txt"
[1] "A01Basp_EPRSc2_conf0.4_5052_AcarbonariusITEM501

In [ ]:
if (exists("Asp_IS_META")){rm("Asp_IS_META")}
profvis({
for (file in list.files(DIR_IN, pattern = "[0-9]_.*_R(1|2)(pos|neg)_TLEN_EndMotif.txt")){
    print(file)
    sample_tmp <- unlist(strsplit(file, "_"))[1] 
    taxID_tmp <- unlist(strsplit(file, "_"))[4]
    mapGenome_tmp <- unlist(strsplit(file, "_"))[5]
    side_tmp <- unlist(strsplit(file, "_"))[6]
    
    if(countLines(paste0(DIR_IN, file)) > 1){
        pre_tmp_IS <- fread(paste0(DIR_IN, file), sep = ",") %>% 
                    dplyr::rename(EndMotif_tmp = EndMotif) %>% 
                    mutate(sample_id = sample_tmp) %>% 
                    mutate(side = side_tmp) %>% 
                    mutate(taxID = taxID_tmp) %>% 
                    mutate(mapGenome = mapGenome_tmp) %>% 
                    mutate(read = substr(side_tmp, 0, 2)) %>% 
                    filter(TLEN != 0) %>% #remove TLEN not equal to 0
                    filter(abs(TLEN) < 1000) %>% #filter max TLEN of 1000
                    filter(!grepl(EndMotif_tmp, pattern = "N")) %>% #remove reads with end-motif including "N"
                    group_by(TLEN, EndMotif_tmp, sample_id, side, taxID, mapGenome) %>% 
                    summarise(Count = sum(Count), .groups = "keep")
        
        if (grepl(side_tmp, pattern = "neg")){
            tmp_IS <- pre_tmp_IS 
            tmp_IS$EndMotif <- as.character(reverseComplement(DNAStringSet(pre_tmp_IS$EndMotif_tmp))) 
        } else {
            tmp_IS <- pre_tmp_IS 
            tmp_IS$EndMotif <- pre_tmp_IS$EndMotif_tmp
        }
        
        tmp_IS_meta <- merge(x = samples_metadata, y = tmp_IS, by.x = "sample_id", by.y = "sample_id") 
        if (!exists("Asp_IS_META")){Asp_IS_META <- tmp_IS_meta} else {
            Asp_IS_META <- rbind(Asp_IS_META, tmp_IS_meta)}
    }
    if (exists("Asp_IS_META")){
        fwrite(Asp_IS_META, "../../output/PP_figures/Asp_IS_meta.csv", row.names = FALSE)
    }
}
})

In [ ]:
if (exists("unmappedHost_IS_META")){rm("unmappedHost_IS_META")}
profvis({
for (file in list.files(DIR_IN, pattern = "___.*_R(1|2)(pos|neg)_TLEN_EndMotif.txt")){
    sample_tmp <- unlist(strsplit(file, "_"))[1] 
    taxID_tmp <- unlist(strsplit(file, "_"))[4]
    mapGenome_tmp <- unlist(strsplit(file, "_"))[5]
    side_tmp <- unlist(strsplit(file, "_"))[6]
    
    if(countLines(paste0(DIR_IN, file)) > 1){
        pre_tmp_IS <- fread(paste0(DIR_IN, file), sep = ",") %>% 
                    dplyr::rename(EndMotif_tmp = EndMotif) %>% 
                    mutate(sample_id = sample_tmp) %>% 
                    mutate(side = side_tmp) %>% 
                    mutate(taxID = taxID_tmp) %>% 
                    mutate(mapGenome = mapGenome_tmp) %>% 
                    mutate(read = substr(side_tmp, 0, 2)) %>% 
                    filter(TLEN != 0) %>% #remove TLEN not equal to 0
                    filter(abs(TLEN) < 1000) %>% #filter max TLEN of 1000
                    filter(!grepl(EndMotif_tmp, pattern = "N")) %>% #remove reads with end-motif including "N"
                    group_by(TLEN, EndMotif_tmp, sample_id, side, taxID, mapGenome) %>% 
                    summarise(Count = sum(Count), .groups = "keep")
        
        if (grepl(side_tmp, pattern = "neg")){
            tmp_IS <- pre_tmp_IS 
            tmp_IS$EndMotif <- as.character(reverseComplement(DNAStringSet(pre_tmp_IS$EndMotif_tmp))) 
        } else {
            tmp_IS <- pre_tmp_IS 
            tmp_IS$EndMotif <- pre_tmp_IS$EndMotif_tmp
        }
        
        tmp_IS_meta <- merge(x = samples_metadata, y = tmp_IS, by.x = "sample_id", by.y = "sample_id") 
        if (!exists("unmappedHost_IS_META")){unmappedHost_IS_META <- tmp_IS_meta} else {
            unmappedHost_IS_META <- rbind(unmappedHost_IS_META, tmp_IS_meta)}
    }
    if (exists("unmappedHost_IS_META")){
        fwrite(unmappedHost_IS_META, "../../output/PP_figures/unmappedHost_IS_meta.csv", row.names = FALSE)
    }
}
})

In [ ]:
Asp_IS_META$taxID %>% unique()
Asp_IS_META %>% 
    mutate(read = substr(side, 1,2)) %>% 
    group_by(sample_id, sample_type, taxID, read, mapGenome) %>% 
    summarise(sumCount = sum(Count), .groups = "keep") %>% 
    filter(read == "R1") %>% 
    mutate(taxID = 
           ifelse(taxID == "1131492", yes = "k2_Aspergillaceae", no = 
           ifelse(taxID == "5052", yes = "k2_Aspergillus", no = 
           ifelse(taxID == "746128", yes = "k2_Aspergillus_fumigatus", no = 
           ifelse(taxID == "", yes = "unmappedHost", no = taxID))))) %>% 
    #filter(mapGenome == "AfumigatusA1163" ) %>%
    mutate(taxID = ifelse(taxID == "", yes = "unmappedHost", no = taxID)) %>% 
    reshape2::dcast(formula = sample_id + sample_type + read + mapGenome ~ taxID, value.var = "sumCount")

In [ ]:
Asp_IS_META %>% 
    mutate(read = substr(side, 1,2)) %>% 
    mutate(TLEN = abs(TLEN)) %>% 
    group_by(sample_id, sample_type, taxID, read, mapGenome, TLEN) %>% 
    summarise(sumCount = sum(Count), .groups = "keep") %>% 
    filter(read == "R1") %>% 
    filter(mapGenome == "AfumigatusA1163" ) %>%
    mutate(taxID = 
           ifelse(taxID == "1131492", yes = "k2_Aspergillaceae", no = 
           ifelse(taxID == "5052", yes = "k2_Aspergillus", no = 
           ifelse(taxID == "746128", yes = "k2_Aspergillus_fumigatus", no = 
           ifelse(taxID == "", yes = "unmappedHost", no = taxID))))) %>% 
    ggplot(aes(x = TLEN, y = sumCount, group = taxID, col = taxID)) + 
        geom_segment(aes(x = TLEN, xend = TLEN, y = 0, yend = sumCount)) + 
        facet_grid(rows = vars(taxID), cols = vars(sample_id)) + theme_bw()